In [5]:
import pandas as pd
import gzip

In [4]:
timeval = ['time', 'value']
files = ['data/heatpump_temperature_1.csv.gz', 'data/heatpump_temperature_2.csv.gz', 'data/heatpump_temperature_3.csv.gz', 'data/heatpump_temperature_4.csv.gz', 'data/heatpump_temperature_5.csv.gz', 'data/heatpump_temperature_6.csv.gz', 'data/heatpump_return_external.csv.gz', 'data/heatpump_hot_gas.csv.gz',  'data/heatpump_probe_in.csv.gz', 'data/heatpump_mk1.csv.gz', 'data/heatpump_mk1_reference.csv.gz', 'data/heatpump_starts_compressor1.csv.gz', 'data/heatpump_state.csv.gz', 'data/heatpump_extended_state.csv.gz']
cols = ['outside', 'return', 'reference_return', 'supply', 'servicewater_reference', 'servicewater', 'return_external', 'hot_gas', 'probe_in', 'mk1', 'mk1_reference', 'starts_compressor1', 'state', 'extended_state']

for i, file in enumerate(files):
    with gzip.open(file, "rb") as f:
        df = pd.read_csv(f, sep=';', names=timeval)

#    df = pd.read_csv(file, sep=';', names=timeval)
    df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d %H:%M:%S')
    df.sort_values('time', inplace=True)
    
    df.rename(columns={"value": cols[i]}, inplace=True)

    print(file)
    print(df)

    if i == 0:
        merged_df = df
    else:
        merged_df = pd.merge_asof(merged_df.sort_values('time'), df, on='time')

merged_df.set_index('time', inplace=True)

print(merged_df)

#print(temp1)
#print(return_external)

FileNotFoundError: [Errno 2] No such file or directory: 'data/heatpump_temperature_1.csv'

In [35]:
merged_df['compressor_heating'] = merged_df['extended_state'].str.contains('compressor heating up:') | merged_df['extended_state'].str.contains('unknown:')
merged_df['defrosting'] = merged_df['extended_state'].str.contains('defrosting:')
merged_df['heatpump_running'] = merged_df['state'].str.contains('Heatpump running:')
merged_df['heatpump_heating'] = merged_df['extended_state'].str.contains('heating:')
merged_df['heatpump_servicewater'] = merged_df['extended_state'].str.contains('service water:')


In [36]:
df_defrosting = merged_df[merged_df['defrosting']==True]
#df_defrosting.groupby(by=[df_defrosting.index.month, df_defrosting.index.year])['defrosting'].sum()
df_defrosting.groupby(by=[df_defrosting.index.month])['hot_gas'].mean()

time
1     49.268421
2     47.779328
3     45.437250
4     43.195951
5     41.610417
10    41.817818
11    44.976126
12    47.408741
Name: hot_gas, dtype: float64

In [38]:
df_night = merged_df[merged_df.index.hour == 22]
print(df_night.groupby(by=[df_night.index.week])['hot_gas'].mean())
print(df_night.groupby(by=[df_night.index.week])['compressor_heating'].sum())



time
1     14.433574
2     15.800000
3     14.075399
4     15.525028
5     17.141829
6     14.537316
7     15.548469
8     15.317804
9     15.924297
10    16.616212
11    16.805267
12    17.097427
13    18.615747
14    19.477051
15    20.537031
16    20.374599
17    20.067411
18    21.521059
19    23.241848
20    22.583965
21    26.299360
22    26.805753
23    25.746602
24    26.052708
25    27.751928
26    28.099902
27    28.113965
28    27.284555
29    28.818042
30    29.028017
31    28.782789
32    27.727880
33    27.712985
34    27.895511
35    27.132382
36    24.837901
37    24.407562
38    23.862086
39    23.326798
40    20.134608
41    21.702313
42    20.926358
43    19.357206
44    18.276557
45    17.362019
46    17.124287
47    17.672892
48    14.656778
49    18.794903
50    17.470110
51    17.442094
52    16.304890
Name: hot_gas, dtype: float64
time
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8     16
9      0
10     0
11    10
12     0
13    18
14    29
15

In [21]:
merged_df

,outside,return,reference_return,supply,servicewater_reference,servicewater,return_external,hot_gas,probe_in,mk1,mk1_reference,starts_compressor1,state,extended_state,compressor_heating,defrosting,heatpump_running,heatpump_heating,heatpump_servicewater
time,,,,,,,,,,,,,,,,,,,
2016-01-16 14:35:50,13.8,NaN,NaN,34.0,NaN,NaN,NaN,60.9,3.7,NaN,NaN,1447,Heatpump running: 04:30:27,heating: 04:30:27,False,False,True,True,False
2016-01-16 14:36:00,13.9,NaN,NaN,34.1,NaN,38.5,NaN,61.0,3.7,NaN,NaN,1447,Heatpump running: 04:30:38,heating: 04:30:38,False,False,True,True,False
2016-01-16 14:36:50,13.8,28.9,NaN,34.3,NaN,38.6,NaN,61.0,3.6,NaN,NaN,1447,Heatpump running: 04:31:17,heating: 04:31:17,False,False,True,True,False
2016-01-16 14:37:01,13.9,28.9,NaN,34.3,NaN,38.6,NaN,61.0,3.6,NaN,NaN,1447,Heatpump running: 04:31:37,heating: 04:31:37,False,False,True,True,False
2016-01-16 14:37:51,13.8,28.9,NaN,34.2,NaN,38.6,NaN,61.1,3.6,33.1,NaN,1447,Heatpump running: 04:32:28,heating: 04:32:28,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24 22:57:30,11.9,21.7,15.0,21.7,43.0,42.4,38.2,14.9,5.9,22.1,0.0,3745,Heatpump stopped: 08:13:56,standby: 08:13:56,False,False,False,False,False
2020-12-24 23:03:14,12.0,21.6,15.0,21.7,43.0,42.4,38.2,14.8,5.6,22.1,0.0,3745,Heatpump stopped: 08:19:37,standby: 08:19:37,False,False,False,False,False
2020-12-24 23:03:24,11.9,21.6,15.0,21.7,43.0,42.4,38.2,14.8,5.6,22.1,0.0,3745,Heatpump stopped: 08:19:47,standby: 08:19:47,False,False,False,False,False
